<a href="https://colab.research.google.com/github/LeeDongHun38/recommendation_LLM/blob/main/cold_start_b3d2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Step 2. Cold-start 시나리오 실험**

🎯 **목표**

사용자 정보 없이, 순수하게 영화의 콘텐츠(줄거리, 리뷰 텍스트)가 얼마나 유사한지를 기반으로 비슷한 영화를 추천한다. 즉, 기본적으로는 "이 영화와 비슷한 분위기의 다른 영화"를 추천하는 방식이 된다.


🎯 **사용 데이터**

*   all_embeddings.npy (계산용 핵심 데이터)
*   asin_to_index.pkl (조회 및 결과 해석용)
*   index_to_asin.pkl (조회 및 결과 해석용)

🎯 **예상 결과물**

특정 영화 asin을 입력했을 때, 그 영화와 임베딩 벡터가 가장 유사한 다른 영화 asin 목록.

**세부 실행 계획**

상대적으로 무거운 모델인 코사인 유사도 측정을 통해 추천 구현 -> 이후 가벼운 모델인 FAISS 를 사용해, 둘의 성능 비교

성능이 비슷하다면 (추천 목록이 유사하면) 상대적 가벼운 모델인 FAISS를 사용, 경량화까지 의의를 둔다

# **2.1 코사인 유사도를 통한 Cold-start 추천 구현**

**코사인 유사도 (Cosine Similarity)**

두 벡터 사이의 **'각도'**를 측정하여, 두 벡터가 얼마나 같은 방향을 가리키는지를 계산합니다. 결과는 -1에서 1 사이의 값으로 나오며, 1에 가까울수록 유사도가 높다는 의미입니다.

**왜 사용하는가?**

SentenceTransformer와 같은 임베딩 모델은 의미가 비슷한 문장을 벡터 공간에서 가까운 방향을 가리키도록 만듭니다. 따라서 두 영화 임베딩 벡터의 '방향'이 얼마나 비슷한지를 측정하는 코사인 유사도는, 두 영화의 '콘텐츠 의미'가 얼마나 유사한지를 측정하는 매우 효과적이고 직관적인 척도가 됩니다.

**장점**


*   정확성: 모든 벡터와 일일이 비교하므로(Brute-force), 이론적으로 가장 정확한 유사도 상위 N개를 찾을 수 있습니다.
*   직관성: 원리가 간단하고 이해하기 쉽습니다.


**단점**

*   속도: 데이터가 많아질수록 계산량이 정비례하여 늘어납니다. 영화 1편과 나머지 26,000편을 비교하는 것은 괜찮지만, 만약 100만 편이 된다면 속도가 매우 느려져 실제 서비스에 사용하기 어렵습니다.




In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import time

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 저장된 데이터 로드

input_dir = '/content/drive/MyDrive/데사경영학술제/data/'
df = pd.read_pickle(input_dir + 'final_movies_df.pkl')
print("데이터 로드 완료.")

데이터 로드 완료.


In [ ]:
# 리뷰 단위 데이터 -> 영화 단위 데이터 변환

'''
이유: 기존 df를 그대로 활용했더니, 리뷰의 유사도를 보고 판단해, 같은 영화들을 추천해줬다.
즉, 다른 사람이 같은 영화를 보고 남긴 리뷰가 유사하니, 비슷한 영화로 판단해 같은 영화를 추천하는 문제 발생함.
따라서 기존 리뷰 단위로 구성되어 있던 데이터를 영화단위로 수정하는 전처리 작업을 추가했다. 이는 같은 영화들을 그룹화해 평균 임베딩을 내는 방식으로 구현.
'''


# np.vstack을 사용하여 모든 임베딩을 하나의 큰 배열로 쌓은 후, 각 임베딩이 올바른 형태로 변환되도록 list comprehension을 사용

embeddings_array = np.array([np.array(e) for e in df['embedding']])
df['embedding_vectors'] = list(embeddings_array)


# asin을 기준으로 그룹화하고, 각 영화의 대표 임베딩(평균)과 제목을 추출
# as_index=False를 통해 그룹화한 asin을 열로 유지

movie_df = df.groupby('asin', as_index=False).agg(
    title=('title', 'first'),   # 첫 번째 제목을 대표 제목으로 사용
    description=('description', 'first'),    # description도 필요하다면 추가
    embedding=('embedding_vectors', lambda x: np.mean(np.vstack(x), axis=0))  # 임베딩 벡터들의 평균을 계산하여 대표 벡터로 삼음
)

In [ ]:
# just for check

print(len(movie_df))
print(movie_df.head())

# 총 6613개의 행 존재, 즉 데이터셋에는 영화가 총 6613개 존재했다.

6613
         asin                                              title  \
0  B0002V7U1Y  Larry Bird: A Basketball Legend, 25th Annivers...   
1  B00062IWNY        Chair Dancing Fitness ?Through The Decades?   
2  B0007ZA5EE  Cathe Friedrich Body Blast Kickboxing Kick, Pu...   
3  B000812R20  Cathe Friedrich's Intensity Series: Pyramid Up...   
4  B000GDBOPQ                                         South Park   

                                         description  \
0  ['The complete DVD guide to Bird watching. Cel...   
1  ['Exercise ... in Your Chair!Chair Dancing Fit...   
2  ["Cathe Friedrich's Kick, Punch and Crunch + L...   
3  ["Cathe Friedrich's Pyramid Lower + Upper Body...   
4  ['This is the season that started it all! Join...   

                                           embedding  
0  [0.028488921, 0.023676472, -0.013285047, -0.11...  
1  [-0.0049788035, -0.09142853, -0.025349244, -0....  
2  [0.05669291, -0.049910106, -0.047095552, 0.048...  
3  [-0.001709284, 0.019171046

In [ ]:
# 영화 단위 데이터 기준, 추천 시스템 준비

all_movie_embeddings = np.array(movie_df['embedding'].tolist())   # 해당 영화에 대해 모든 사용자가 남긴 후기의 평균 임베딩 값.
movie_asin_list = movie_df['asin'].tolist()   # all_movie_embeddings에 일치하는 영화 asin 값
movie_asin_to_index = {asin: i for i, asin in enumerate(movie_asin_list)}   # 영화 asin을 입력하면, 그 영화가 all_movie_embeddings에 몇 번째 행에 있는지 알려주는 딕셔너리
movie_index_to_asin = {i: asin for i, asin in enumerate(movie_asin_list)}   # 행렬의 인덱스 (행 번호)를 입력하면, 그 위치에 어떤 asin이 있는지 알려주는 딕셔너리

print(f"총 {len(movie_df)}개의 영화에 대한 변수 생성 완료.")

총 6613개의 영화에 대한 변수 생성 완료.


In [ ]:
# 추천 함수 정의

def cos_recs(target_asin, top_n=10):
    try:
        target_idx = movie_asin_to_index[target_asin]   # target_asin을 입력받으면, 만들어둔 딕셔너리로 all_movie_embeddings의 몇 번째 행에 존재하는지 파악해 저장
        target_vector = all_movie_embeddings[target_idx].reshape(1, -1)   # 해당 번째 행에 존재하는 벡터를 가져옴

        sim_scores = cosine_similarity(target_vector, all_movie_embeddings)[0]    # 해당 영화의 벡터 (targtet_vector와 다른 모든 영화 벡터 간의 유사도 계산)
        sim_indices = np.argsort(sim_scores)[::-1]    # 유사도 점수가 높은 순으로 반환.

        top_indices = sim_indices[1:top_n+1]
        top_scores = sim_scores[top_indices]

        recommended_asins = [movie_index_to_asin[i] for i in top_indices]   # 추천된 영화 목록을 딕셔너리를 사용해 다시 asin 형태로 반환함

        # 추천 결과를 movie_df에서 찾아 반환
        recs_df = movie_df[movie_df['asin'].isin(recommended_asins)].copy()
        recs_df['similarity'] = top_scores

        # 추천된 순서대로 정렬
        recs_ordered = recs_df.set_index('asin').loc[recommended_asins].reset_index()

        return recs_ordered[['asin', 'title', 'similarity', 'description']]

    except KeyError:
        return f"'{target_asin}'에 해당하는 영화를 찾을 수 없습니다."   # 만약 딕셔너리에 없는 asin이 들어오면, KeyError 출력

In [ ]:
# 함수 실행 및 결과 확인

example_asin = movie_df['asin'].iloc[0]
# expample_asin = input('asin을 하나 입력해주세요: ')
movie_title = movie_df[movie_df['asin'] == example_asin]['title'].iloc[0]

print(f"\n--- '{movie_title}' (asin: {example_asin})와(과) 비슷한 영화 추천 결과 ---")

start_time = time.time()
recommendation_result = cos_recs(example_asin, top_n=5)
end_time = time.time()
elapsed_time = end_time - start_time

print(recommendation_result)
print(f"\n코사인 유사도 계산 소요 시간: {elapsed_time:.4f}초")


--- 'Larry Bird: A Basketball Legend, 25th Anniversary Edition [DVD]' (asin: B0002V7U1Y)와(과) 비슷한 영화 추천 결과 ---
         asin                    title  similarity  \
0  B000HZEE64             Coach Carter    0.478220   
1  B001MQT2VO               Fast Break    0.404486   
2  B09C9S6S4N  Space Jam: A New Legacy    0.392270   
3  B00A2YAEIW             The Birdcage    0.403521   
4  B07P5X99P5   In Search of Greatness    0.399368   

                                         description  
0  ['A successful businessman returns to his high...  
1  ['A New York delicatessen worker talks his way...  
2  ['When LeBron James and his son are trapped in...  
3  ['The son of gay club owner and his drag star ...  
4  ['IN SEARCH OF GREATNESS is a groundbreaking d...  

코사인 유사도 계산 소요 시간: 0.0123초


**결과 분석 및 평가**



---



**긍정적인 측면 (모델이 잘 작동하고 있다는 증거)**

1. 높은 주제적 연관성: 추천된 5개의 영화 중 4개는 타겟 영화인 '래리 버드' 다큐멘터리와 매우 높은 연관성을 가집니다.

* Coach Carter, Hoosiers, Space Jam: A New Legacy: 모두 **농구(Basketball)**라는 핵심 주제를 공유합니다. 모델이 '농구 영화'라는 클러스터를 성공적으로 찾아냈음을 의미합니다.
* In Search of Greatness: 스포츠 다큐멘터리라는 점에서 '래리 버드' 다큐멘터리와 장르적, 주제적 유사성이 매우 높습니다.
* 이는 우리가 생성한 임베딩이 영화의 핵심 콘텐츠와 주제를 잘 파악하고 있음을 보여주는 강력한 증거입니다.

2. 상대적 순위의 합리성: 진지한 농구 드라마인 '코치 카터'가 가장 높은 유사도(0.478)를 기록한 것은 매우 합리적인 결과입니다. 이는 모델이 단순 키워드 매칭을 넘어, 콘텐츠의 톤앤매너까지 어느 정도 감지하고 있음을 시사합니다.


---



**분석이 필요한 이상치 (Outlier)**

'The Birdcage' 추천: 농구 레전드 '래리 버드(Larry Bird)'에 대한 추천 결과로, 농구와 전혀 관련 없는 코미디 영화 '버드케이지(The Birdcage)'가 포함된 점은 분석이 필요한 부분입니다.

예상 원인: 'Bird'라는 단어의 영향력

가장 가능성 높은 원인은 임베딩 모델이 두 영화의 제목과 내용에 공통적으로 포함된 'Bird'라는 키워드에 과도한 가중치를 부여했을 수 있다는 것입니다.
SentenceTransformer와 같은 모델은 문맥을 이해하지만, 때로는 이처럼 강력하고 독특한 키워드가 전체적인 의미 판단에 예상보다 큰 영향을 미치기도 합니다. 이는 텍스트 기반 추천 모델이 가진 본질적인 한계 중 하나를 보여주는 흥미로운 사례입니다.

# **2.2 FAISS를 이용한 Cold-start 추천 구현**

FAISS의 목표는 코사인 유사도와 **'거의 동일한 결과'**를 내면서도, 검색 '속도'를 수십, 수백 배 이상 향상시키는 것입니다. 이를 위해 FAISS는 먼저 모든 벡터 데이터를 '인덱스(Index)'라는 특별한 구조로 만들어 둡니다. 이 인덱스는 데이터를 효율적으로 검색할 수 있도록 미리 정리해놓은 '색인'과 같습니다.

인덱스 내에서 바로 추천이 가능하므로, 속도가 향상되는 구조.

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 81.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import pickle
import faiss
import time

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 저장된 데이터 로드 (기존 2.1과 같은 구조)

input_dir = '/content/drive/MyDrive/데사경영학술제/data/'
movie_df = pd.read_pickle(input_dir + 'final_movies_df.pkl').groupby('asin', as_index=False).agg(
    title=('title', 'first'),
    description=('description', 'first'),
    embedding=('embedding', lambda x: np.mean([np.array(e) for e in x], axis=0))
)

all_movie_embeddings = np.array(movie_df['embedding'].tolist(), dtype='float32')
movie_asin_list = movie_df['asin'].tolist()
movie_asin_to_index = {asin: i for i, asin in enumerate(movie_asin_list)}
movie_index_to_asin = {i: asin for i, asin in enumerate(movie_asin_list)}

In [ ]:
# FAISS 인덱스 생성


# 벡터의 차원 가져옴
d = all_movie_embeddings.shape[1]

# FAISS 인덱스를 초기화. IndexFlatIP는 Inner Product를 사용
# 벡터가 정규화(normalized)된 경우, 내적은 코사인 유사도와 동일한 역할을 수행
index = faiss.IndexFlatIP(d)

# 따라서 벡터를 정규화하는 과정이 꼭 필요함
faiss.normalize_L2(all_movie_embeddings)

# 정규화된 임베딩 벡터들을 인덱스에 추가
index.add(all_movie_embeddings)
print(f"인덱스 생성 완료. 총 {index.ntotal}개의 벡터가 인덱싱되었습니다.")

인덱스 생성 완료. 총 6613개의 벡터가 인덱싱되었습니다.


In [ ]:
# FAISS 추천 함수 정의

def faiss_recs(target_asin, top_n=10):
    try:
        # 타겟 영화의 벡터 검색
        target_idx = movie_asin_to_index[target_asin]
        target_vector = all_movie_embeddings[target_idx].reshape(1, -1)

        # FAISS 인덱스 상에서 유사한 아이템을 검색
        distances, indices = index.search(target_vector, top_n + 1)   # (유사도 점수 배열, 인덱스 배열)을 반환

        # 검색된 인덱스와 점수를 추출 (자기 자신 제외)
        top_indices = indices[0][1:]
        top_scores = distances[0][1:]

        # 인덱스를 asin으로 변환
        recommended_asins = [movie_index_to_asin[i] for i in top_indices]

        # 결과 df 생성
        recs_df = pd.DataFrame({'asin': recommended_asins, 'similarity': top_scores})
        meta_info = movie_df[['asin', 'title', 'description']]
        final_recs = pd.merge(recs_df, meta_info, on='asin', how='left')

        return final_recs

    except KeyError:
        return f"'{target_asin}'에 해당하는 영화를 찾을 수 없습니다."

In [ ]:
# 함수 실행 및 시간 측정

example_asin = movie_df['asin'].iloc[0]
# expample_asin = input('asin을 하나 입력해주세요: ')
movie_title = movie_df[movie_df['asin'] == example_asin]['title'].iloc[0]

print(f"\n--- '[FAISS] {movie_title}' (asin: {example_asin})와(과) 비슷한 영화 추천 결과 ---")

start_time = time.time()
recommendation_result_faiss = faiss_recs(example_asin, top_n=5)
end_time = time.time()
elapsed_time_faiss = end_time - start_time

print(recommendation_result_faiss)
print(f"\nFAISS 검색 소요 시간: {elapsed_time_faiss:.4f}초")


--- '[FAISS] Larry Bird: A Basketball Legend, 25th Anniversary Edition [DVD]' (asin: B0002V7U1Y)와(과) 비슷한 영화 추천 결과 ---
         asin  similarity                    title  \
0  B000HZEE64    0.478220             Coach Carter   
1  B001MQT2VO    0.404486               Fast Break   
2  B09C9S6S4N    0.403521  Space Jam: A New Legacy   
3  B00A2YAEIW    0.399368             The Birdcage   
4  B07P5X99P5    0.392270   In Search of Greatness   

                                         description  
0  ['A successful businessman returns to his high...  
1  ['A New York delicatessen worker talks his way...  
2  ['When LeBron James and his son are trapped in...  
3  ['The son of gay club owner and his drag star ...  
4  ['IN SEARCH OF GREATNESS is a groundbreaking d...  

FAISS 검색 소요 시간: 0.0123초
